In [1]:
import findspark

In [2]:
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
from pyspark.ml.feature import StringIndexer

In [6]:
spark=SparkSession.builder.appName('CruiseShip').getOrCreate()

23/07/06 19:53:11 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/06 19:53:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 19:53:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/06 19:53:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/06 19:53:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
from pyspark.ml.regression import LinearRegression

In [8]:
data=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [9]:
indexer=StringIndexer(inputCols=["Cruise_line"],outputCols=["Cruise_line_indexed"])

In [12]:
indexed=indexer.fit(data).transform(data)


In [13]:
data.show()


+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [14]:
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.5

In [15]:
indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_indexed: double (nullable = false)



In [16]:
from pyspark.ml.linalg import Vectors

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_indexed']

In [23]:
assembler = VectorAssembler(
    inputCols=[
 'Age',
'Cruise_line_indexed',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density'],
    outputCol="features")

In [26]:
output=assembler.transform(indexed)

In [27]:
final_data=output.select('features','crew')

In [28]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,16.0,30.2769...|3.55|
|[6.0,16.0,30.2769...|3.55|
|[26.0,1.0,47.262,...| 6.7|
|[11.0,1.0,110.0,2...|19.1|
|[17.0,1.0,101.353...|10.0|
|[22.0,1.0,70.367,...| 9.2|
|[15.0,1.0,70.367,...| 9.2|
|[23.0,1.0,70.367,...| 9.2|
|[19.0,1.0,70.367,...| 9.2|
|[6.0,1.0,110.2389...|11.5|
|[10.0,1.0,110.0,2...|11.6|
|[28.0,1.0,46.052,...| 6.6|
|[18.0,1.0,70.367,...| 9.2|
|[17.0,1.0,70.367,...| 9.2|
|[11.0,1.0,86.0,21...| 9.3|
|[8.0,1.0,110.0,29...|11.6|
|[9.0,1.0,88.5,21....|10.3|
|[15.0,1.0,70.367,...| 9.2|
|[12.0,1.0,88.5,21...| 9.3|
|[20.0,1.0,70.367,...| 9.2|
+--------------------+----+
only showing top 20 rows



In [29]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [30]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              107|
|   mean|7.737570093457955|
| stddev|3.462648359636506|
|    min|             0.59|
|    max|             19.1|
+-------+-----------------+



In [31]:
lr=LinearRegression(labelCol='crew')

In [33]:
lr_model=lr.fit(train_data)

23/07/06 19:59:32 WARN Instrumentation: [3698240c] regParam is zero, which might cause numerical instability and overfitting.
23/07/06 19:59:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/06 19:59:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [35]:
test_results=lr_model.evaluate(test_data)

In [36]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.27704675806330314|
| -1.4080232698496786|
| -0.1250406161180475|
|-0.31924434302832694|
|-0.04754685622055099|
| -0.9491834279963065|
|-0.05779778074047748|
|  -0.664709484741989|
|-0.15186453064168326|
| -0.5368776558097839|
| -0.6362876081391953|
| 0.12977112377487643|
|    2.05859908471456|
| -0.3025808564048855|
|-0.27040355916095926|
| -0.8991491285423852|
|  0.7828383500264398|
| -1.1574949163854225|
|  0.6995988296649891|
|  0.9043181832475486|
+--------------------+
only showing top 20 rows



In [37]:
test_results.rootMeanSquaredError

0.71532217959136

In [38]:
test_results.r2


0.960164193858265

In [39]:
unlabeled_data=test_data.select('features')

In [40]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[4.0,0.0,220.0,54...|
|[5.0,3.0,86.0,21....|
|[5.0,7.0,133.5,39...|
|[6.0,0.0,158.0,43...|
|[6.0,2.0,113.0,37...|
|[6.0,16.0,30.2769...|
|[9.0,2.0,116.0,26...|
|[9.0,5.0,105.0,27...|
|[10.0,2.0,91.6270...|
|[10.0,5.0,86.0,21...|
|[10.0,5.0,105.0,2...|
|[10.0,8.0,77.0,20...|
|[10.0,12.0,151.4,...|
|[11.0,1.0,86.0,21...|
|[12.0,0.0,90.09,2...|
|[12.0,0.0,138.0,3...|
|[12.0,1.0,88.5,21...|
|[12.0,1.0,88.5,21...|
|[12.0,4.0,77.104,...|
|[12.0,6.0,91.0,20...|
+--------------------+
only showing top 20 rows



In [41]:
predictions=lr_model.transform(unlabeled_data)

In [42]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,0.0,220.0,54...|20.722953241936697|
|[5.0,3.0,86.0,21....| 9.408023269849679|
|[5.0,7.0,133.5,39...|13.255040616118048|
|[6.0,0.0,158.0,43...|13.919244343028327|
|[6.0,2.0,113.0,37...|12.047546856220551|
|[6.0,16.0,30.2769...| 4.499183427996306|
|[9.0,2.0,116.0,26...|11.057797780740477|
|[9.0,5.0,105.0,27...|11.344709484741989|
|[10.0,2.0,91.6270...| 9.151864530641683|
|[10.0,5.0,86.0,21...| 9.736877655809783|
|[10.0,5.0,105.0,2...|11.316287608139195|
|[10.0,8.0,77.0,20...| 8.870228876225124|
|[10.0,12.0,151.4,...| 10.47140091528544|
|[11.0,1.0,86.0,21...| 9.602580856404886|
|[12.0,0.0,90.09,2...| 8.950403559160959|
|[12.0,0.0,138.0,3...|12.749149128542385|
|[12.0,1.0,88.5,21...|  9.50716164997356|
|[12.0,1.0,88.5,21...|10.457494916385423|
|[12.0,4.0,77.104,...|  8.89040117033501|
|[12.0,6.0,91.0,20...| 9.085681816752452|
+--------------------+------------

In [44]:
from pyspark.sql.functions import corr

In [45]:
data.select(corr('crew','cabins')).show()


+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [46]:
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

